In [2]:
import sqlite3
import pandas as pd

# Connect to db
connection = sqlite3.connect('analytics_track.db')
cursor = connection.cursor()

In [3]:
def run_query(qry):
    return pd.read_sql_query(qry, connection)

Tables:
* event_codes (Event_Msg_Type, Action_Type, Event_Msg_Type_Description, Action_Type_Description)
* game_lineup (Game_id,Period, Person_id, Team_id, status)
* play_by_play (Game_id, Event_Num, Event_Msg_Type, Period, WC_Time, PC_Time, Action_Type, Option1, Option2, Option3, Team_id, Person1, Person2,Team_id_type)

## Explore data from first game: 
Game_id = 021fd159b55773fba8157e2090fe0fe2

In [3]:
game1_lineup = run_query("SELECT * from game_lineup WHERE Game_id = '021fd159b55773fba8157e2090fe0fe2'")
game1_lineup.sort_values(by = ['Team_id', 'Period']).drop('index', axis=1).head()

,Game_id,Period,Person_id,Team_id,status
0,021fd159b55773fba8157e2090fe0fe2,1,881f83d2dee3f18c7d1751659406144e,012059d397c0b7e5a30a5bb89c0b075e,A
3,021fd159b55773fba8157e2090fe0fe2,1,cec898a1d355dbfbad8c760615fde1af,012059d397c0b7e5a30a5bb89c0b075e,A
5,021fd159b55773fba8157e2090fe0fe2,1,a99f44bbff39e352191a870e17f04537,012059d397c0b7e5a30a5bb89c0b075e,A
7,021fd159b55773fba8157e2090fe0fe2,1,89706b99ddd00dc05d37ef5cafc04276,012059d397c0b7e5a30a5bb89c0b075e,A
8,021fd159b55773fba8157e2090fe0fe2,1,2b313e2bcef0268bc8e9415132ba9997,012059d397c0b7e5a30a5bb89c0b075e,A


In [102]:
qry = """
SELECT Period, PC_Time, WC_Time, Event_Num, p.Event_Msg_Type, p.Action_Type, Event_Msg_Type_Description,
       Action_Type_Description, Option1, Option2, Team_id, Person1, Person2
FROM play_by_play p 
INNER JOIN event_codes e ON p.Event_Msg_Type = e.Event_Msg_Type AND p.Action_Type = e.Action_Type
WHERE p.Game_id = '021fd159b55773fba8157e2090fe0fe2'
ORDER BY Period asc, PC_Time desc, WC_Time asc, Event_Num asc
"""
game1_plays = run_query(qry)
game1_plays.head()

,Period,PC_Time,WC_Time,Event_Num,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description,Option1,Option2,Team_id,Person1,Person2
0,1,7200,546427,0,12,0,Start Period,,0,0,1473d70e5646a26de3c52aa1abd85b1f,6bcf6c1f8c373d25fca1579bc4464a91,6bcf6c1f8c373d25fca1579bc4464a91
1,1,7200,546495,1,10,0,Jump Ball,,0,0,012059d397c0b7e5a30a5bb89c0b075e,89706b99ddd00dc05d37ef5cafc04276,307beab25b1021a548b4a47550bc4b25
2,1,7050,546665,2,2,1,Missed Shot,Jump Shot,3,0,012059d397c0b7e5a30a5bb89c0b075e,cec898a1d355dbfbad8c760615fde1af,6bcf6c1f8c373d25fca1579bc4464a91
3,1,6960,546714,3,4,0,Rebound,Unknown,0,0,012059d397c0b7e5a30a5bb89c0b075e,307beab25b1021a548b4a47550bc4b25,6bcf6c1f8c373d25fca1579bc4464a91
4,1,6920,546886,6,6,4,Foul,Offensive,0,0,cff694c8186a4bd377de400e4f60fe47,c00264c3114d23bac482e9de50fb7d28,89706b99ddd00dc05d37ef5cafc04276


## Notes about Data

* Event associated with neither team at the start of every game:
    * Event_Msg_type = 12, which signals start of the game.
    * Team_id_type = 0 & Team_id = 1473d70e5646a26de3c52aa1abd85b1f
    
* None values for Player Column:
    * Player_id = 6bcf6c1f8c373d25fca1579bc4464a91	

In [5]:
# first = run_query("SELECT * from play_by_play WHERE Team_id_type = 0")
# first = run_query("SELECT * from play_by_play WHERE Team_id = '1473d70e5646a26de3c52aa1abd85b1f'")
# first

# Game Class & Game Simulator:

In [103]:
class Game:
           
    # Initialize a new game object with: GameID, & TeamID's
    def __init__(self, game_id, team1_id, team2_id):
        self.game = game_id
        self.team1 = team1_id
        self.team2 = team2_id 
        self.team1_score = 0
        self.team2_score = 0

        # Dictionary with Player_id:Active Status
        self.team1_active = {}
        self.team2_active = {}

        # Dictionary with Player_id:Plus/Minus
        self.team1_pm = {}
        self.team2_pm = {}

    ### Update Plus/Minus stats and score:  ###
    
    def update_score(self, team_id, points):
        # Plus/Minus points relative to TEAM 1
        if team_id == self.team1:
            self.team1_score += points
            pm = points
        elif team_id == self.team2:
            self.team2_score += points
            pm = -points
            
        for player, active in self.team1_active.items():
            if active:
                try:
                    self.team1_pm[player] += pm
                except KeyError:
                    self.team1_pm[player] = pm

        for player, active in self.team2_active.items():
            if active:
                try:
                    self.team2_pm[player] -= pm
                except KeyError:
                    self.team2_pm[player] = -pm           

    ### Updating active players ###       
    
    def get_starter(self, team_id, period):
        ### Gets the starting 5 for each quarter ###
        ### Returns iterable ###
        df = run_query("SELECT Person_id FROM game_lineup WHERE Game_id='{}' AND Team_id='{}' AND Period={}".format(self.game,team_id,period))
        return df.Person_id
    
    def start_period(self, period):
        tm1_starters = self.get_starter(self.team1, period)
        tm2_starters = self.get_starter(self.team2, period)
        for player in tm1_starters:
            self.team1_active[player] = True
        for player in tm2_starters:
            self.team2_active[player] = True
            
    def end_period(self):
        for player in self.team1_active:
            self.team1_active[player] = False
        for player in self.team2_active:
            self.team2_active[player] = False            
        
    def make_substitution(self,leaving, entering):
        ### Changes the active state of the leaving and entering players.
        ### Checks the active players sets instead of TeamID provided because
        ### original dataset is inconsistent for that field.
        if leaving in self.team1_active.keys():
            self.team1_active[leaving] = False
            self.team1_active[entering] = True
        elif leaving in self.team2_active.keys():
            self.team2_active[leaving] = False
            self.team2_active[entering] = True
        else:
            print('ERROR: Player', leaving, "not found on either team found.")

    def get_plusminus(self):
        return [self.team1_pm, self.team2_pm]

### Query to get play-by-play for a specific GameID:

In [104]:
plays_query = """
SELECT Period, PC_Time, WC_Time, Event_Num, p.Event_Msg_Type, p.Action_Type, Event_Msg_Type_Description,
       Action_Type_Description, Option1, Option2, Team_id, Person1, Person2
FROM play_by_play p 
INNER JOIN event_codes e 
    ON p.Event_Msg_Type = e.Event_Msg_Type 
    AND p.Action_Type = e.Action_Type
WHERE p.Game_id = '{}'
ORDER BY Period asc, PC_Time desc, WC_Time asc, Event_Num asc;
"""

### Game simulator that reads the table above and updates the stats:

In [105]:
from collections import deque

def game_simulation(game_id):
    teams_list = run_query("SELECT distinct Team_id from game_lineup WHERE Game_id = '{}'".format(game_id)).Team_id
    game = Game(game_id, teams_list[0], teams_list[1])
    play_by_play = run_query(plays_query.format(game_id))
    
    # To keep track of fouls/and subs for special case:
    foul_state = False
    foul_time = 0
    sub_queue = deque([])
    
    for play in play_by_play.itertuples():
        event_type = play.Event_Msg_Type
                       
        if foul_state:
            if event_type not in [3,8,9] and play.PC_Time != foul_time:  # If action after foul is not a free throw or substitution
                while len(sub_queue) > 0:   # make the substitutions (after PM was updated)
                    sub = sub_queue.popleft()
                    game.make_substitution(sub[0], sub[1])
#                     print('... making retroactive sub ...', sub[0], 'for', sub[1])
                foul_state = False
            elif event_type == 8:   # Substitution
                # Store sub that needs to be made in queue for after foul/freethrow sequence
                sub_queue.append((play.Person1, play.Person2))
#                 print('... saving sub ...', play.Person1, 'for', play.Person2)
                continue
                
        if event_type in [2, 4, 5, 7, 9, 10, 11]:
        # Ignore Missed Shot, Rebound, Turnover, Violation, Timeout, JumpBall
#             print(play.Event_Msg_Type_Description,':', play.Action_Type_Description, play.Team_id)
            continue  # exit this iteration of for-loop
            
        elif event_type == 12:  # Start Period
            game.start_period(play.Period)
            print('Start period', play.Period)
            
        elif event_type == 13:  # End Period
            game.end_period()
            print('End of Period', play.Period)
            print('.......... Score:', game.team1_score, '-', game.team2_score)
            
        elif event_type == 1:  # Made Shot
            game.update_score(play.Team_id, play.Option1)
#             print('Team', play.Team_id, 'scored', play.Option1, 'points. Score:', game.team1_score, '-', game.team2_score)
            
        elif event_type == 3:  # Free Throw
            if play.Option1 == 1:
                game.update_score(play.Team_id, play.Option1)
#                 print(play.Event_Msg_Type_Description, 'made Score:', game.team1_score, '-', game.team2_score)
#             else:
#                 print(play.Event_Msg_Type_Description, 'missed Score:', game.team1_score, '-', game.team2_score)
                
        elif event_type == 6:  # Foul
            foul_state = True
            foul_time = play.PC_Time
#             print('Foul :',play.Action_Type_Description)

        elif event_type == 8:  # Substitution
            game.make_substitution(play.Person1, play.Person2)
#             print('Substitute - Leaving:', play.Person1,', Entering:',play.Person2)
            
#         elif event_type == 11:  # Ejection
#             print('Ejection: Player', play.Person1)
            
#         elif event_type == 9:
#             print('Timeout: Team', play.Team_id)?
            
    return game

In [106]:
g1 = game_simulation('021fd159b55773fba8157e2090fe0fe2')

Start period 1
End of Period 1
.......... Score: 29 - 16
Start period 2
End of Period 2
.......... Score: 57 - 42
Start period 3
End of Period 3
.......... Score: 79 - 68
Start period 4
End of Period 4
.......... Score: 93 - 91


In [107]:
g2 = game_simulation('03a31e84b194d6c8a2eab5d70ba67acf')

Start period 1
End of Period 1
.......... Score: 24 - 27
Start period 2
End of Period 2
.......... Score: 51 - 55
Start period 3
End of Period 3
.......... Score: 78 - 93
Start period 4
End of Period 4
.......... Score: 108 - 123


In [108]:
g3 = game_simulation('06bb1d31c63891e2580ff12e4e6505b4')

Start period 1
End of Period 1
.......... Score: 36 - 28
Start period 2
End of Period 2
.......... Score: 66 - 60
Start period 3
End of Period 3
.......... Score: 92 - 92
Start period 4
End of Period 4
.......... Score: 121 - 116


In [109]:
g3.get_plusminus()

[{'394e78a51b6d2fd3c046f11f49fcab57': 3,
  '1a8dfc8c85278dd28ed7985e40fc6018': -5,
  'd8e4f0fd2e836518e681c99a3f9e4411': -6,
  '1a48480fc3370082483f158f10948bba': -10,
  '646b0273682c1592a938bbfb21332ded': -2,
  '1c73194e0fa711701298e4a27b046bab': 2,
  '762b12c3cb57fad4b882040b3293fa31': 15,
  '3056d865481cd5919672eb92e1f8b53f': 7,
  '85b8db431f11d51d5def87fc58229be4': 10,
  '1532445b96ca1edd411bfae064b189bb': 11},
 {'adcd53acc9f5585db4c510745b2bac89': -5,
  '9d4117dd37b6a8592a711ef940b82de4': -3,
  '58f273dbbf002f0ec411fc247e07dd0a': -10,
  '9f0ee42268a2542d7fd028fe79184286': -5,
  '1cd0599b37f77c2be76a2c6df23d1ba8': 3,
  'eaa6c5d9e964e04fa05b4db476fb46f0': 2,
  'b098111d3935607a714ddc7d4b057a1b': -4,
  '28ce9a11a56397b259299fdd9710dab5': -2,
  'f285f13f0c036a88bd85535e47e88aef': -1}]

In [110]:
game_to_table(g3)

,Game_ID,Player_ID,Player_Plus/Minus
0,06bb1d31c63891e2580ff12e4e6505b4,394e78a51b6d2fd3c046f11f49fcab57,3
1,06bb1d31c63891e2580ff12e4e6505b4,1a8dfc8c85278dd28ed7985e40fc6018,-5
2,06bb1d31c63891e2580ff12e4e6505b4,d8e4f0fd2e836518e681c99a3f9e4411,-6
3,06bb1d31c63891e2580ff12e4e6505b4,1a48480fc3370082483f158f10948bba,-10
4,06bb1d31c63891e2580ff12e4e6505b4,646b0273682c1592a938bbfb21332ded,-2
5,06bb1d31c63891e2580ff12e4e6505b4,1c73194e0fa711701298e4a27b046bab,2
6,06bb1d31c63891e2580ff12e4e6505b4,762b12c3cb57fad4b882040b3293fa31,15
7,06bb1d31c63891e2580ff12e4e6505b4,3056d865481cd5919672eb92e1f8b53f,7
8,06bb1d31c63891e2580ff12e4e6505b4,85b8db431f11d51d5def87fc58229be4,10
9,06bb1d31c63891e2580ff12e4e6505b4,1532445b96ca1edd411bfae064b189bb,11


## Tests to make sure functions are working:

In [80]:
print(g1.team1_score, g1.team2_score)

93 91


In [82]:
g1.get_plusminus()

[{'881f83d2dee3f18c7d1751659406144e': -3,
  'cec898a1d355dbfbad8c760615fde1af': -2,
  'a99f44bbff39e352191a870e17f04537': 20,
  '89706b99ddd00dc05d37ef5cafc04276': 11,
  '2b313e2bcef0268bc8e9415132ba9997': -11,
  'd6bc5eb6ed54c734628cb4d237f99031': 1,
  '1362d9ad805e7cc4ea3a9467c91142d6': -3,
  'b2fada91a0c8a0f400623c425286ae2e': 6,
  '2d9594e4d250679e7a3383ad5ccd2f44': 0,
  '314c71574689f752c17b83cd6a4d770e': -9},
 {'27ea17a8685c4919f157e83fe9cb2d9e': -2,
  '57bbd7e30bc694aeee9ee40c583e6811': -9,
  '33963fe856a1523ff46438ba07d1d99f': 7,
  'c00264c3114d23bac482e9de50fb7d28': -2,
  '307beab25b1021a548b4a47550bc4b25': -3,
  '603daefb2fa321f3abab289805fdcc25': -9,
  '60b5f9f68ccbbf7ba3fa13a2189e2775': 0,
  'c1fe6ed491d04e3cdf50755dff23b263': 9,
  '8bb18fb1d0cc10c985afa823ecec6d18': -1}]

## Convert Game object to output table:

In [85]:
def game_to_table(game):
    gid = game.game
    tm1 = pd.DataFrame([[gid, player, pm] for player,pm in game.team1_pm.items()], columns = ['Game_ID', 'Player_ID', 'Player_Plus/Minus'])
    tm2 = pd.DataFrame([[gid, player, pm] for player,pm in game.team2_pm.items()], columns = ['Game_ID', 'Player_ID', 'Player_Plus/Minus'])
    return pd.concat([tm1, tm2])

In [ ]:
game_ids = run_query('SELECT DISTINCT Game_id FROM game_lineup').Game_id

In [115]:
i = 1
output_table = pd.DataFrame()
for gid in game_ids:
    print('Gettings stats for game', i)
    sim = game_simulation(gid)
    output_table = pd.concat([output_table,game_to_table(sim)])
    i += 1


Gettings stats for game 0
Start period 1
End of Period 1
.......... Score: 29 - 16
Start period 2
End of Period 2
.......... Score: 57 - 42
Start period 3
End of Period 3
.......... Score: 79 - 68
Start period 4
End of Period 4
.......... Score: 93 - 91
Gettings stats for game 1
Start period 1
End of Period 1
.......... Score: 24 - 27
Start period 2
End of Period 2
.......... Score: 51 - 55
Start period 3
End of Period 3
.......... Score: 78 - 93
Start period 4
End of Period 4
.......... Score: 108 - 123
Gettings stats for game 2
Start period 1
End of Period 1
.......... Score: 36 - 28
Start period 2
End of Period 2
.......... Score: 66 - 60
Start period 3
End of Period 3
.......... Score: 92 - 92
Start period 4
End of Period 4
.......... Score: 121 - 116
Gettings stats for game 3
Start period 1
End of Period 1
.......... Score: 26 - 23
Start period 2
End of Period 2
.......... Score: 59 - 56
Start period 3
End of Period 3
.......... Score: 86 - 89
Start period 4
End of Period 4
......

Start period 1
End of Period 1
.......... Score: 27 - 27
Start period 2
End of Period 2
.......... Score: 51 - 48
Start period 3
End of Period 3
.......... Score: 87 - 75
Start period 4
End of Period 4
.......... Score: 116 - 101
Gettings stats for game 34
Start period 1
End of Period 1
.......... Score: 30 - 30
Start period 2
End of Period 2
.......... Score: 54 - 59
Start period 3
End of Period 3
.......... Score: 81 - 76
Start period 4
End of Period 4
.......... Score: 120 - 116
Gettings stats for game 35
Start period 1
End of Period 1
.......... Score: 28 - 31
Start period 2
End of Period 2
.......... Score: 48 - 59
Start period 3
End of Period 3
.......... Score: 70 - 88
Start period 4
End of Period 4
.......... Score: 96 - 115
Gettings stats for game 36
Start period 1
End of Period 1
.......... Score: 25 - 22
Start period 2
End of Period 2
.......... Score: 56 - 59
Start period 3
End of Period 3
.......... Score: 78 - 87
Start period 4
End of Period 4
.......... Score: 114 - 113


In [118]:
output_table.to_csv('Berkeley_Ballers_Q1_BBALL.csv', index=False)
